# TUGAS PRAKTIKUM - THE KNIGHT'S TOUR

## Deskripsi Masalah

Jika sebuah bidak kuda diletakkan pada sebarang kotak untuk kemudian melakukan perjalanan (dengan cara pergerakan kuda) mengunjungi ke semua (8 × 8) kotak papan catur.

Jika diinginkan situasi bahwa kuda tsb dapat:
- **a. Mengakhiri perjalanan di sebarang kotak (open tour)**
- **b. Mengakhiri perjalanan pada attacking square (closed tour)**

---

## Algoritma

Menggunakan **Backtracking** dengan **Warnsdorff's Heuristic**:
- Backtracking: Mencoba semua kemungkinan pergerakan dan mundur jika menemui jalan buntu
- Warnsdorff's Heuristic: Prioritaskan kotak yang memiliki lebih sedikit pilihan gerakan selanjutnya (mengurangi jalan buntu)

## Import Libraries

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle
import time

## Implementasi Knight's Tour Class

In [ ]:
class KnightsTour:
    def __init__(self, board_size=8, closed_tour=False):
        """
        Initialize Knight's Tour solver
        
        Parameters:
        - board_size: ukuran papan (default 8x8)
        - closed_tour: True untuk closed tour, False untuk open tour
        """
        self.size = board_size
        self.closed_tour = closed_tour
        self.board = [[-1 for _ in range(board_size)] for _ in range(board_size)]
        
        # Pergerakan kuda (L-shaped: 2 kotak satu arah, 1 kotak tegak lurus)
        # Urutan: kanan-atas, atas-kanan, atas-kiri, kiri-atas, kiri-bawah, bawah-kiri, bawah-kanan, kanan-bawah
        self.moves_x = [2, 1, -1, -2, -2, -1, 1, 2]
        self.moves_y = [1, 2, 2, 1, -1, -2, -2, -1]
        
        self.solution_time = 0
        
    def is_safe(self, x, y):
        """
        Cek apakah posisi valid dan belum dikunjungi
        
        Returns:
        - True jika posisi dalam papan dan belum dikunjungi
        """
        return (0 <= x < self.size and 
                0 <= y < self.size and 
                self.board[x][y] == -1)
    
    def count_unvisited_neighbors(self, x, y):
        """
        Hitung tetangga yang belum dikunjungi (Warnsdorff's heuristic)
        
        Returns:
        - Jumlah kotak yang dapat dikunjungi dari posisi (x, y)
        """
        count = 0
        for i in range(8):
            next_x = x + self.moves_x[i]
            next_y = y + self.moves_y[i]
            if self.is_safe(next_x, next_y):
                count += 1
        return count
    
    def solve_tour(self, start_x=0, start_y=0):
        """
        Selesaikan Knight's Tour menggunakan backtracking dengan Warnsdorff's heuristic
        
        Parameters:
        - start_x: posisi x awal (baris)
        - start_y: posisi y awal (kolom)
        
        Returns:
        - True jika solusi ditemukan, False jika tidak
        """
        start_time = time.time()
        
        # Reset papan
        self.board = [[-1 for _ in range(self.size)] for _ in range(self.size)]
        self.board[start_x][start_y] = 0
        
        result = self._solve_util(start_x, start_y, 1, start_x, start_y)
        
        self.solution_time = time.time() - start_time
        return result
    
    def _solve_util(self, x, y, move_count, start_x, start_y):
        """
        Fungsi rekursif untuk backtracking dengan Warnsdorff's heuristic
        """
        # Jika semua kotak telah dikunjungi
        if move_count == self.size * self.size:
            # Untuk closed tour, cek apakah kuda bisa kembali ke start
            if self.closed_tour:
                for i in range(8):
                    if (x + self.moves_x[i] == start_x and 
                        y + self.moves_y[i] == start_y):
                        return True
                return False
            return True
        
        # Dapatkan semua gerakan yang mungkin dan urutkan berdasarkan Warnsdorff's heuristic
        moves = []
        for i in range(8):
            next_x = x + self.moves_x[i]
            next_y = y + self.moves_y[i]
            if self.is_safe(next_x, next_y):
                priority = self.count_unvisited_neighbors(next_x, next_y)
                moves.append((priority, next_x, next_y))
        
        # Urutkan berdasarkan prioritas (lebih sedikit tetangga yang belum dikunjungi = prioritas lebih tinggi)
        moves.sort()
        
        # Coba setiap gerakan
        for _, next_x, next_y in moves:
            self.board[next_x][next_y] = move_count
            
            if self._solve_util(next_x, next_y, move_count + 1, start_x, start_y):
                return True
            
            # Backtrack jika tidak berhasil
            self.board[next_x][next_y] = -1
        
        return False
    
    def visualize(self, save_fig=False, filename='knights_tour.png'):
        """
        Visualisasi Knight's Tour
        
        Parameters:
        - save_fig: jika True, simpan gambar
        - filename: nama file untuk menyimpan gambar
        """
        if self.board[0][0] == -1:
            print("Tidak ada solusi yang ditemukan!")
            return
        
        fig, ax = plt.subplots(figsize=(12, 12))
        
        # Buat pola papan catur
        for i in range(self.size):
            for j in range(self.size):
                # Warna kotak catur
                color = '#F0D9B5' if (i + j) % 2 == 0 else '#B58863'
                ax.add_patch(Rectangle((j, self.size-1-i), 1, 1, 
                                      facecolor=color, edgecolor='black', linewidth=1))
                
                # Tambahkan nomor urutan gerakan
                if self.board[i][j] != -1:
                    ax.text(j + 0.5, self.size - 1 - i + 0.5, 
                           str(self.board[i][j]),
                           ha='center', va='center', 
                           fontsize=10, fontweight='bold',
                           color='black', bbox=dict(boxstyle='round,pad=0.3', 
                                                   facecolor='white', alpha=0.7))
        
        # Gambar jalur pergerakan
        path = []
        for move in range(self.size * self.size):
            for i in range(self.size):
                for j in range(self.size):
                    if self.board[i][j] == move:
                        path.append((j + 0.5, self.size - 1 - i + 0.5))
        
        if path:
            path_x, path_y = zip(*path)
            ax.plot(path_x, path_y, 'r-', linewidth=2.5, alpha=0.5, label='Path')
            
            # Tandai posisi awal
            ax.plot(path_x[0], path_y[0], 'go', markersize=15, label='Start', zorder=5)
            
            # Tandai posisi akhir
            ax.plot(path_x[-1], path_y[-1], 'bs', markersize=15, label='End', zorder=5)
            
            # Untuk closed tour, hubungkan akhir ke awal
            if self.closed_tour:
                ax.plot([path_x[-1], path_x[0]], [path_y[-1], path_y[0]], 
                       'r--', linewidth=2.5, alpha=0.5, label='Return to Start')
        
        ax.set_xlim(0, self.size)
        ax.set_ylim(0, self.size)
        ax.set_aspect('equal')
        ax.axis('off')
        ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
        
        tour_type = "Closed Tour" if self.closed_tour else "Open Tour"
        plt.title(f"Knight's Tour ({tour_type}) - {self.size}x{self.size} Board\nSolution Time: {self.solution_time:.4f}s", 
                 fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        
        if save_fig:
            plt.savefig(filename, dpi=300, bbox_inches='tight')
            print(f"Gambar disimpan sebagai {filename}")
        
        plt.show()
    
    def print_board(self):
        """
        Cetak papan di console
        """
        if self.board[0][0] == -1:
            print("Tidak ada solusi yang ditemukan!")
            return
        
        tour_type = "CLOSED TOUR" if self.closed_tour else "OPEN TOUR"
        print(f"\n{tour_type} - Knight's Tour Solution:")
        print("=" * (self.size * 4 + 1))
        
        for i in range(self.size):
            for j in range(self.size):
                print(f"{self.board[i][j]:3d}", end=" ")
            print()
        print("=" * (self.size * 4 + 1))
        print(f"Waktu solusi: {self.solution_time:.4f} detik")
    
    def get_path_coordinates(self):
        """
        Dapatkan koordinat jalur dalam urutan gerakan
        
        Returns:
        - List of tuples (x, y) merepresentasikan jalur
        """
        path = []
        for move in range(self.size * self.size):
            for i in range(self.size):
                for j in range(self.size):
                    if self.board[i][j] == move:
                        path.append((i, j))
        return path

## Situasi A: Open Tour

Kuda dapat mengakhiri perjalanan di sebarang kotak (tidak perlu kembali ke posisi awal).

In [ ]:
print("SITUASI A: OPEN TOUR")
print("=" * 60)
print("Kuda dapat mengakhiri perjalanan di sebarang kotak\n")

# Buat instance untuk open tour
knight_open = KnightsTour(board_size=8, closed_tour=False)

# Selesaikan tour mulai dari kotak (0, 0)
print("Mencari solusi dari posisi (0, 0)...")
if knight_open.solve_tour(start_x=0, start_y=0):
    print("✓ Solusi ditemukan untuk Open Tour!\n")
    knight_open.print_board()
else:
    print("✗ Tidak ada solusi ditemukan untuk Open Tour")

In [ ]:
# Visualisasi Open Tour
knight_open.visualize(save_fig=False)

## Situasi B: Closed Tour

Kuda harus mengakhiri perjalanan pada attacking square (kotak yang dapat kembali ke posisi awal dengan satu gerakan).

In [ ]:
print("SITUASI B: CLOSED TOUR")
print("=" * 60)
print("Kuda harus mengakhiri perjalanan pada kotak yang dapat kembali ke awal\n")

# Buat instance untuk closed tour
knight_closed = KnightsTour(board_size=8, closed_tour=True)

# Selesaikan tour mulai dari kotak (0, 0)
print("Mencari solusi dari posisi (0, 0)...")
if knight_closed.solve_tour(start_x=0, start_y=0):
    print("✓ Solusi ditemukan untuk Closed Tour!\n")
    knight_closed.print_board()
else:
    print("✗ Tidak ada solusi ditemukan untuk Closed Tour")

In [ ]:
# Visualisasi Closed Tour
knight_closed.visualize(save_fig=False)

## Eksperimen dengan Posisi Awal yang Berbeda

In [ ]:
# Coba dengan posisi awal yang berbeda untuk Open Tour
print("Eksperimen: Open Tour dari posisi (3, 3) - tengah papan")
print("=" * 60)

knight_center = KnightsTour(board_size=8, closed_tour=False)
if knight_center.solve_tour(start_x=3, start_y=3):
    print("✓ Solusi ditemukan!\n")
    knight_center.print_board()
    knight_center.visualize()
else:
    print("✗ Tidak ada solusi ditemukan")

## Analisis Performa untuk Berbagai Ukuran Papan

In [ ]:
# Analisis performa untuk papan berbeda ukuran
print("Analisis Performa untuk Berbagai Ukuran Papan (Open Tour)")
print("=" * 70)

results = []
board_sizes = [5, 6, 7, 8]

for size in board_sizes:
    print(f"\nTesting {size}x{size} board...")
    knight = KnightsTour(board_size=size, closed_tour=False)
    
    if knight.solve_tour(start_x=0, start_y=0):
        results.append((size, knight.solution_time, True))
        print(f"  ✓ Solusi ditemukan dalam {knight.solution_time:.4f} detik")
    else:
        results.append((size, 0, False))
        print(f"  ✗ Tidak ada solusi")

print("\n" + "=" * 70)
print("\nRingkasan:")
print("-" * 70)
print(f"{'Ukuran Papan':<15} {'Status':<15} {'Waktu (detik)':<15}")
print("-" * 70)
for size, time_taken, success in results:
    status = "Berhasil" if success else "Gagal"
    time_str = f"{time_taken:.4f}" if success else "N/A"
    print(f"{size}x{size:<12} {status:<15} {time_str:<15}")
print("=" * 70)

## Visualisasi Perbandingan Open vs Closed Tour

In [ ]:
# Perbandingan visual side-by-side
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))

# Fungsi helper untuk menggambar papan
def draw_board(ax, knight, title):
    if knight.board[0][0] == -1:
        ax.text(0.5, 0.5, 'Tidak ada solusi', 
               ha='center', va='center', transform=ax.transAxes)
        return
    
    size = knight.size
    
    for i in range(size):
        for j in range(size):
            color = '#F0D9B5' if (i + j) % 2 == 0 else '#B58863'
            ax.add_patch(Rectangle((j, size-1-i), 1, 1, 
                                  facecolor=color, edgecolor='black', linewidth=1))
            
            if knight.board[i][j] != -1:
                ax.text(j + 0.5, size - 1 - i + 0.5, 
                       str(knight.board[i][j]),
                       ha='center', va='center', 
                       fontsize=9, fontweight='bold',
                       color='black', bbox=dict(boxstyle='round,pad=0.3', 
                                               facecolor='white', alpha=0.7))
    
    path = []
    for move in range(size * size):
        for i in range(size):
            for j in range(size):
                if knight.board[i][j] == move:
                    path.append((j + 0.5, size - 1 - i + 0.5))
    
    if path:
        path_x, path_y = zip(*path)
        ax.plot(path_x, path_y, 'r-', linewidth=2, alpha=0.5)
        ax.plot(path_x[0], path_y[0], 'go', markersize=12, label='Start')
        ax.plot(path_x[-1], path_y[-1], 'bs', markersize=12, label='End')
        
        if knight.closed_tour:
            ax.plot([path_x[-1], path_x[0]], [path_y[-1], path_y[0]], 
                   'r--', linewidth=2, alpha=0.5)
    
    ax.set_xlim(0, size)
    ax.set_ylim(0, size)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.set_title(title, fontsize=14, fontweight='bold', pad=15)
    ax.legend()

# Gambar kedua tour
draw_board(ax1, knight_open, f'Open Tour\n(Waktu: {knight_open.solution_time:.4f}s)')
draw_board(ax2, knight_closed, f'Closed Tour\n(Waktu: {knight_closed.solution_time:.4f}s)')

plt.suptitle('Perbandingan Open Tour vs Closed Tour', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

## Kesimpulan

### Hasil Implementasi:

1. **Open Tour**: Kuda berhasil mengunjungi semua 64 kotak pada papan 8×8 tanpa harus kembali ke posisi awal.

2. **Closed Tour**: Kuda berhasil mengunjungi semua 64 kotak dan dapat kembali ke posisi awal dengan satu gerakan kuda.

### Algoritma yang Digunakan:

- **Backtracking**: Mencoba semua kemungkinan pergerakan secara sistematis
- **Warnsdorff's Heuristic**: Optimisasi dengan memilih kotak yang memiliki paling sedikit pilihan gerakan selanjutnya, mengurangi kemungkinan jalan buntu

### Kompleksitas:

- **Kompleksitas Waktu**: O(8^(n²)) dalam worst case, tetapi Warnsdorff's heuristic secara signifikan mengurangi waktu eksekusi dalam praktik
- **Kompleksitas Ruang**: O(n²) untuk menyimpan papan

### Catatan:

- Closed tour lebih sulit ditemukan dibanding open tour karena constraint tambahan
- Posisi awal mempengaruhi waktu pencarian solusi
- Untuk papan berukuran kecil (< 5×5), solusi mungkin tidak ada